In [1]:
import numpy
import os

In [2]:
filepath = os.path.split(os.path.realpath('__file__'))[0]

In [3]:
#reading dictionary of words 
with open(os.path.join(filepath,'Tagalog Words.txt'), 'r', encoding='utf-8') as f:
    word_list = [words.strip() for words in f.readlines()]

In [4]:
#Function to calculate Levenshtein Distance
dist_dict = {}

def LevDistance(orig_string, corr_string):
    '''
    Function to calculate Levenshtein Distance
    orig_string : Original string which needs to be corrected
    corr_string : Potential corrected version of the string
    '''

    if min(len(orig_string), len(corr_string)) == 0:
        #If one either of the string is empty, distance is the length
        #of the non-empty string
        return max(len(orig_string), len(corr_string))
    elif (orig_string==corr_string):
        #If both the strings are exactly same, distance is zero
        return 0
    
    if orig_string[-1] == corr_string[-1]:
        cost = 0
    else:
        cost = 1
#     print(orig_string," + ",corr_string, " : cost - ",str(cost))   
    
    l1 = (orig_string[:-1], corr_string)
    if not l1 in dist_dict:
        dist_dict[l1] = LevDistance(*l1)
    
    l2 = (orig_string, corr_string[:-1])
    if not l2 in dist_dict:
        dist_dict[l2] = LevDistance(*l2)

    l3 = (orig_string[:-1], corr_string[:-1])
    if not l3 in dist_dict:
        dist_dict[l3] = LevDistance(*l3)
        
    res = min([dist_dict[l1]+1, dist_dict[l2]+1, dist_dict[l3]+cost])
        
    return res

In [5]:
LevDistance("Babuy","Baboy")

1

In [7]:
import os
import random
# from edit_distance import EditDistance
import re
import time

In [14]:
class SpellChecker:
    def __init__(self,text_path):
        dir_name = os.path.dirname(os.path.realpath('__file__'))
        self.dictionary_file = dir_name+"/Tagalog Words.txt"
        self.corpus_file = dir_name+text_path
        pass

    def read_dictionary(self):
        self.dictionary_words = set(line.strip() for line in open(self.dictionary_file))
        
    def corpus_word_freq(self, print_results=False, ignore_higfreq_words = True, eval_lines =50):
        '''
        Function to generate dictionary for correct and corrupted words in the corpus.
        
        Generating dictionary for correct words in the corpus by comparing if they
        exists in the available dictionary 'corpus_word_dict' of words and if they 
        don't, adding them the to the dictionary 'invalid_corpus_tokens' of 
        corrupted words.
        '''
        # initiating dictionary for words
        self.corpus_word_dict = {}
        self.invalid_corpus_tokens = {}
        
        lines_eval = 0
        
        with open(self.corpus_file,'r') as f:
            for line in f:
                word_list = re.findall(r'\b[a-zA-Z]+\b', line)
                for word in word_list:
                    #checking if the word exists in the given dictionary
                    if word.lower() in self.dictionary_words:
                        self.corpus_word_dict[word.lower()] = self.corpus_word_dict.get(word.lower(), 0) + 1
                    # if it doesn't exists, assuming it to be corrupted word and adding to other dictionary  
                    # However, if the words start with a Upper case alphabet, assuming it to be a proper noun and 
                    # ignoring those words to be added to corrupted dict
                    elif(lines_eval < eval_lines and not (word[0].isupper() and word[1].islower())):    
                        self.invalid_corpus_tokens[word.lower()] = self.invalid_corpus_tokens.get(word.lower(),0) + 1
                lines_eval += 1        

        '''
        Since some words won't be the part of correct words dict, and if a word 
        in corrupted dict appears over a certain number of times, it is 
        considered as not corrupted and moved to the correct word dict.
        '''
        if ignore_higfreq_words:                
            self.ignored_words = set()    
            for key,val in self.invalid_corpus_tokens.items():
                if int(val) > 25:
                    self.ignored_words.add(key)
            for word in self.ignored_words:
                self.corpus_word_dict[word] = self.invalid_corpus_tokens[word]
                del self.invalid_corpus_tokens[word]
            
        if print_results:
            print(str(len(self.corpus_word_dict))," correct words added to the corpus dictionary")
            print(str(len(self.invalid_corpus_tokens))," corrupted words exists in the corpus")

        
    def print_top_n_line(self, n = 50, ignore_higfreq_words= True, calc_word_dict = True):
        '''
        Function to print first n lines from the corpus with underlined 
        corrupted words
        '''
        line_count = 0
        if calc_word_dict:
            self.corpus_word_freq(ignore_higfreq_words = ignore_higfreq_words, eval_lines=n)
            
        with open(self.corpus_file, 'r') as f:
            for line in f:
                word_list = re.findall(r'\b[a-zA-Z]+\b', line)
                for word in word_list:
                    if word.lower() in self.invalid_corpus_tokens:
                        line = re.sub(word,"\033[4m"+word+"\033[0m",line)
                print(line)
                line_count += 1
                if line_count >= n:
                    break
        
        
    def closest_replacement(self, possible_opt_dict):
        '''
        Function to resolve conflict in case of multiple replacements
        with same edit distance available.
        Replacement with the max occurance in the corpus will be selected
        '''
        inv_dict = {}
        for key, value in possible_opt_dict.items():
            if key in self.corpus_word_dict.keys():
                possible_opt_dict[key] = self.corpus_word_dict[key]
                inv_dict[possible_opt_dict[key]] = key
        if len(inv_dict) > 0 :
            return inv_dict[max(inv_dict.keys())]
        else:
            return None 
        
        
    def spell_check_first_n_lines(self, n=50, ignore_higfreq_words=True, min_distance = 4, 
                                  show_status = False, show_options = True, print_results = False):
        '''
        Function to spell check first n lines and replace the corrupted
        words by the closest replacement in the dictionary. 
        
        Closest replacement of the word is defined using Levenshtein Distance
        between the corrupted word and the words in dictonary. If multiple 
        replacement words with shortest distance are found, the replacement option
        with the highest freqeuncy in the corpus is used. 
        
        Output:- Output of the function will be saved in "Output.txt" file in the current directory
        
        n : Number of lines to be evaluated
        ignore_higfreq_words : default:- True; Flag to ignore high frequency word from corrupted word dict
        min_distance : default :- 4; Min edit distance to be used. 
        show_status  : defualt :- False; Flag to enable printing of status with every correction
        show_options : default :- True; Flag to enable printing of options for each corrupted words
        print_results: default :- False; Flag to enable priting of the corrected text after spell check
        '''
        
        self.corpus_word_freq(ignore_higfreq_words = ignore_higfreq_words, eval_lines = n)
        self.replacement_dict = {}
        for index, invalid_word in enumerate(self.invalid_corpus_tokens):
            if show_status:
                print("Processing word "+str(index)+" of "+str(len(self.invalid_corpus_tokens))+" word "+invalid_word)
            distance_dict = {}
            min_dist = min_distance
            for valid_word in self.dictionary_words:
                # Only calculating edit distance with the correct words whose lenght is +/- 1 length of 
                # corrupted words
                if (len(valid_word) >= len(invalid_word)-1) and (len(valid_word) <= len(invalid_word)+1):
                    #edit_dist = EditDistance(invalid_word, valid_word).calculate()
                    edit_dist = LevDistance(valid_word,invalid_word)
                    if edit_dist < min_dist:
                        min_dist = edit_dist
                        distance_dict = {}
                    if edit_dist <= min_dist: 
                        distance_dict[valid_word] =  edit_dist
                        
            #print(distance_dict)
            # In case only one replacement found for the corrupted word with the minimum possible distance
            # then, it will be selected as the replacement option.
            if len(distance_dict) == 1:
                self.replacement_dict[invalid_word] = list(distance_dict.keys())[0]
            # In case multiple replacement options are available with the minimum edit distance, the word 
            # having highest frequency in corpus will be selected as the replacement option.    
            elif len(distance_dict) > 1 and self.closest_replacement(distance_dict) is not None:
                self.replacement_dict[invalid_word] = self.closest_replacement(distance_dict)
            else:    
            # In case no replacement word is found within the minimum distance limit, the corrupted word 
            # will be assigned as the replacement option.    
                self.replacement_dict[invalid_word] = invalid_word
         
        if show_options:
            print("Found the following replacements : ")
            print(self.replacement_dict)
        
        self.correct_text(replace_lines = n, print_res=print_results)
               

    def correct_text(self, replace_lines = 50, print_res=False):
        line_count = 0
        os.remove("Output.txt")
        print("\n")
        with open(self.corpus_file, 'r') as f:
            for line in f:
                word_list = re.findall(r'\b[a-zA-Z]+\b', line)
                for word in word_list:
                    if word.lower() in self.invalid_corpus_tokens:
                        line = re.sub(word,self.replacement_dict[word.lower()],line)
                if print_res:
                    print(line)    
                with open('Output.txt', 'a') as w:        
                    print(line, file=w)
                line_count += 1
                if line_count >= replace_lines:
                    print("\n---Corrected text written to Output.txt---")
                    break

In [22]:
sc = SpellChecker("/test.txt")
sc.read_dictionary()

In [24]:
sc.print_top_n_line(50)

babuy

kamates

seboyas

babai

sanggul

sekwinta

salamen

buhuk

sengkamas

talung

sarewa

kalekasan

pangaku

goneta


In [25]:
sc.corpus_word_freq(eval_lines=500)

In [26]:
starttime = time.time()
sc.spell_check_first_n_lines(500, show_status=True,show_options=False, print_results= True)
print("\n\nExecution time - %s seconds"% round(time.time()-starttime,3))

Processing word 0 of 14 word babuy
Processing word 1 of 14 word kamates
Processing word 2 of 14 word seboyas
Processing word 3 of 14 word babai
Processing word 4 of 14 word sanggul
Processing word 5 of 14 word sekwinta
Processing word 6 of 14 word salamen
Processing word 7 of 14 word buhuk
Processing word 8 of 14 word sengkamas
Processing word 9 of 14 word talung
Processing word 10 of 14 word sarewa
Processing word 11 of 14 word kalekasan
Processing word 12 of 14 word pangaku
Processing word 13 of 14 word goneta


babuy

kamatis

seboyas

babai

sanggol

sekwinta

salamin

buhok

singkamas

talung

sarewa

kalekasan

pangako

goneta


Execution time - 9.354 seconds
